# **SSC0957 - Prática em Ciência de Dados II**
---

### **Alunos:**
> #### Arthur Santorum Lorenzetto - 12559465
> #### André Guarnier De Mitri    - 11395579
> #### Rui Emanuel Lima Vieira    - 11819182

### **Professor:** Alexandre Claudio Botazzo Delbem
---


Este trabalho utilizou dados disponibilizados pelo repositório COVID-19 Data Sharing/BR, disponível em:
https://repositoriodatasharingfapesp.uspdigital.usp.br/".


# 1. Introdução

# Carregando Dados


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

In [2]:
casos = pd.read_parquet('Dados/casos_agrupados.parquet', engine='pyarrow')
casos.head()

,regiao_FIOCRUZ,estado_FIOCRUZ,municipio_FIOCRUZ,coduf_FIOCRUZ,codmun_FIOCRUZ,codRegiaoSaude_FIOCRUZ,nomeRegiaoSaude_FIOCRUZ,data_FIOCRUZ,semanaEpi_FIOCRUZ,populacaoTCU2019_FIOCRUZ,...,pneumopatia_pre,puerpera_pre,sindrome_de_down_pre,estado,QUANTIDADE_TIPO_TRANSPORTE_respiradores_pedido,QUANTIDADE_TIPO_UTI_respiradores_pedido,QUANTIDADE_TIPO_TRANSPORTE_USA_respiradores_pedido,QUANTIDADE_TIPO_TRANSPORTE_respiradores_entrega,QUANTIDADE_TIPO_UTI_respiradores_entrega,QUANTIDADE_TIPO_TRANSPORTE_USA_respiradores_entrega
0,Norte,RO,ALTA FLORESTA D'OESTE,11,110001.0,11005.0,ZONA DA MATA,2020-04-10,15,22945.0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,Norte,RO,ALTA FLORESTA D'OESTE,11,110001.0,11005.0,ZONA DA MATA,2020-04-11,15,22945.0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,Norte,RO,ALTA FLORESTA D'OESTE,11,110001.0,11005.0,ZONA DA MATA,2020-04-12,16,22945.0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,Norte,RO,ALTA FLORESTA D'OESTE,11,110001.0,11005.0,ZONA DA MATA,2020-04-13,16,22945.0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,Norte,RO,ALTA FLORESTA D'OESTE,11,110001.0,11005.0,ZONA DA MATA,2020-04-14,16,22945.0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


# Análise Exploratória Casos e Óbitos

In [3]:
casos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2111030 entries, 0 to 2111029
Data columns (total 67 columns):
 #   Column                                               Dtype         
---  ------                                               -----         
 0   regiao_FIOCRUZ                                       object        
 1   estado_FIOCRUZ                                       object        
 2   municipio_FIOCRUZ                                    object        
 3   coduf_FIOCRUZ                                        int64         
 4   codmun_FIOCRUZ                                       float64       
 5   codRegiaoSaude_FIOCRUZ                               float64       
 6   nomeRegiaoSaude_FIOCRUZ                              object        
 7   data_FIOCRUZ                                         datetime64[ns]
 8   semanaEpi_FIOCRUZ                                    int64         
 9   populacaoTCU2019_FIOCRUZ                             float64       
 10  casosAcumul

In [4]:
def retorna_dados_lista(name,lista_idade,indice_lista):
    if(len(indice_lista[0]) == 0):
        return np.nan
    else:
        return lista_idade[indice_lista]

def df_explode_idade(df,obito=None):
    return df.query("obito_pre == obito_pre") \
                 .assign(index = lambda x: x["obito_pre"].apply(lambda y: np.where(y == obito) if obito is not None
                                                                                           else np.where(y >= 0))) \
                 .assign(idade = lambda x: x.apply(lambda y: retorna_dados_lista(y.name,
                                                                                 y['idade_pre'],
                                                                                 y['index']),
                                                         axis=1),
                        sexo = lambda x: x.apply(lambda y: retorna_dados_lista(y.name,
                                                                                 y['cs_sexo_pre'],
                                                                                 y['index']),
                                                         axis=1)).explode(["idade","sexo"])

# def hist_individuos_contaminados_ORIGINAL(df,title,obito=None):
#     df_explode_idade(df,obito).idade.hist(bins=19)
#     plt.title(title)
#     plt.xlabel("Idade")
#     plt.ylabel("Quantidade de pessoas")
#     plt.show()

def hist_individuos_contaminados(df, title, obito=None):
    df_exploded = df_explode_idade(df, obito)
    fig = px.histogram(
        df_exploded, x="idade", nbins=40, title=title,
        labels={"idade": "Idade"}
    )
    fig.update_layout(yaxis_title="Quantidade de pessoas")
    fig.show()

In [ ]:
hist_individuos_contaminados(casos,"Histograma da idade dos indivíduos contaminados")

In [ ]:
hist_individuos_contaminados(casos,"Histograma da idade dos indivíduos que APENAS se contaminaram no período apurado",0)

In [ ]:
hist_individuos_contaminados(casos,"Histograma da idade dos indivíduos que vieram a falecer no período apurado",1)

In [ ]:
# Criar um gráfico de histograma para o primeiro conjunto de dados
fig1 = px.histogram(df_explode_idade(casos).idade, nbins=19)

# Criar um gráfico de histograma para o segundo conjunto de dados
fig2 = px.histogram(df_explode_idade(casos, 1).idade, nbins=19)

# Criar um subplot com os dois gráficos de histograma
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=["Contaminados", "Óbitos"])
fig.update_layout(title_text= "Contaminados x Óbitos", showlegend=False)
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.update_xaxes(title_text="Idade")
fig.update_yaxes(title_text="Quantidade de pessoas")

# Mostrar o subplot
fig.show()

In [ ]:
df_explode_idade(casos).idade.hist(bins=19)
df_explode_idade(casos,1).idade.hist(bins=19)

plt.title("Contaminados x Mortos")
plt.xlabel("Idade")
plt.ylabel("Quantidade de pessoas")
plt.show()

In [ ]:
casos = casos.assign(nao_morte = lambda x: x["obito_pre"].apply(lambda y: len(np.where(y == 0)[0])),
                             morte = lambda x: x["obito_pre"].apply(lambda y: len(np.where(y == 1)[0])))

def obitos_nao_obitos_contaminados(df):
    return df.melt(id_vars = ["municipio_FIOCRUZ","data_FIOCRUZ"],value_vars = ['nao_morte','morte'],
                                            var_name = 'situacao',value_name='quantidade') \
             .groupby("situacao").quantidade.sum().reset_index() \
             .assign(perc_quantidade = lambda x: 100*x["quantidade"]/x["quantidade"].sum())

obitos_nao_obitos_contaminados(casos)

In [ ]:
 # Casos por município
def volume_casos(df):
    return df.groupby("municipio_FIOCRUZ").casosNovos_FIOCRUZ.sum().sort_values(ascending=False)

volume_casos(casos).head(20)

In [ ]:
def regplot_perc_log(df, x, y):
    df_grouped = df.groupby("municipio_FIOCRUZ").agg({
        'casosNovos_FIOCRUZ': sum,
        'obitosNovos_FIOCRUZ': sum,
        'populacaoTCU2019_FIOCRUZ': 'mean'
    })

    df_grouped['perc_casos'] = 100 * df_grouped['casosNovos_FIOCRUZ'] / df_grouped['populacaoTCU2019_FIOCRUZ']
    df_grouped['perc_obitos'] = 100 * df_grouped['obitosNovos_FIOCRUZ'] / df_grouped['populacaoTCU2019_FIOCRUZ']
    df_grouped['poplog'] = np.log(df_grouped['populacaoTCU2019_FIOCRUZ'])
    df_grouped['perc_obitos_casos'] = 100 * df_grouped['obitosNovos_FIOCRUZ'] / df_grouped['casosNovos_FIOCRUZ']
    df_grouped['casoslog'] = np.log(df_grouped['casosNovos_FIOCRUZ'])
    df_sorted = df_grouped.sort_values('perc_casos')

    fig = px.scatter(df_sorted, x=x, y=y, text=df_sorted.index)
    fig.update_traces(textfont=dict(size=1), marker=dict(size=12))
    fig.update_layout(
        xaxis_title=x,
        yaxis_title=y,
        showlegend=False,
    )

    # Adiciona a linha de regressão
    fig.add_trace(px.scatter(df_sorted, x=x, y=y, trendline="ols").data[1])
    # fig.data[1].line.color = 'red'  # Personalize a cor da linha de regressão

    return fig

print("Por conta da cidade de São Paulo ser um outlier no número de habitantes, precisamos fazer uma transformação para que este problema não seja percebido")
fig = regplot_perc_log(casos, 'perc_casos', 'populacaoTCU2019_FIOCRUZ')
fig.update_layout(
    title='Relação entre a quantidade de casos e população',
    xaxis_title="Percentual de casos",
    yaxis_title="População da cidade",
    showlegend=False,
)

In [ ]:
fig = regplot_perc_log(casos, 'perc_casos', 'poplog')
fig.update_layout(
    title='Relação entre a quantidade de casos e população',
    xaxis_title="Percentual de casos",
    yaxis_title="População da cidade (log)",
    showlegend=False,
)

In [ ]:
fig = regplot_perc_log(casos, 'perc_obitos', 'poplog')
fig.update_layout(
    title='Relação entre a quantidade de óbitos e população',
    xaxis_title="Percentual de óbitos",
    yaxis_title="População da cidade (log)",
    showlegend=False,
)

In [ ]:
fig = regplot_perc_log(casos, 'perc_obitos_casos', 'casoslog')
fig.update_layout(
    title='Relação entre a quantidade de casos e óbitos',
    xaxis_title="Percentual de óbitos",
    yaxis_title="População da cidade (log)",
    showlegend=False,
)

# Plots Comorbidades

In [ ]:
comorbidades = ['asma_pre',
                 'cardiopatia_pre',
                 'diabetes_pre',
                 'doenca_hematologica_pre',
                 'doenca_hepatica_pre',
                 'doenca_neurologica_pre',
                 'doenca_renal_pre',
                 'imunodepressao_pre',
                 'obesidade_pre',
                 'outros_fatores_de_risco_pre',
                 'pneumopatia_pre',
                 'puerpera_pre',
                 'sindrome_de_down_pre']

In [ ]:
def explode_comorbidade_df(df):
    explode = df[["municipio_FIOCRUZ","data_FIOCRUZ","obito_pre","cs_sexo_pre","idade_pre"]+comorbidades] \
                             .explode(["idade_pre",'obito_pre','cs_sexo_pre']+comorbidades).reset_index(drop=True) \
                             .assign(alguma_comorbidade = lambda x: (x[comorbidades]=="sim") \
                                                                           .apply(lambda y: any(y),axis=1),
                                     totalmente_ignorado = lambda x: (x[comorbidades]=="ignorado") \
                                                                       .apply(lambda y: all(y),axis=1)) \
                             .assign(alguma_comorbidade = lambda x: np.where(x["alguma_comorbidade"] == True,"sim",
                                                                          np.where(x["totalmente_ignorado"] == True,
                                                                                    "ignorado",
                                                                                    "não")))
    return explode

casos_explode = explode_comorbidade_df(casos)

In [ ]:
sns.countplot(data = casos_explode,x = "alguma_comorbidade")
plt.title("Qtd de pessoas que se contaminaram com o vírus")
plt.tight_layout()
plt.xlabel("Paciente com alguma comorbidade?")
plt.ylabel("Quantidade de indivíduos contaminados")
plt.show()

In [ ]:
# Nota-se que há uma diferença entre ter alguma comorbidade e os casos com óbito: existe uma relação entre eles
sns.countplot(data = casos_explode.query("obito_pre == 1"),x = "alguma_comorbidade")
plt.title("Qtd de pessoas que morreram e se tinham alguma comorbidade")
plt.xlabel("Paciente com alguma comorbidade?")
plt.ylabel("Quantidade de pacientes que vieram a falecer")
plt.show()

In [ ]:
# Vendo se uma comorbidade sobresai
palette = sns.color_palette("pastel")

sns.countplot(data =
                    casos_explode.query("alguma_comorbidade == 'sim'") \
                                    .melt(id_vars=['municipio_FIOCRUZ','data_FIOCRUZ','obito_pre','idade_pre'],
                                            value_vars=comorbidades,
                                            value_name="existencia_comorbidade",
                                            var_name= "comorbidade") \
                                    .query("existencia_comorbidade == 'sim'"),
              y='comorbidade',
              hue="obito_pre",
              palette=palette)
plt.title("Volume por comorbidades (de óbitos e não óbitos)")
plt.xlabel("Qtd de óbitos")
plt.ylabel("Comorbidade")
plt.show()

In [ ]:
sns.countplot(data = casos_explode.query("obito_pre == 1 & alguma_comorbidade == 'sim'") \
                                    .melt(id_vars=['municipio_FIOCRUZ','data_FIOCRUZ','obito_pre','idade_pre'],
                                            value_vars=comorbidades,
                                            value_name="existencia_comorbidade",
                                            var_name= "comorbidade") \
                                    .query("existencia_comorbidade == 'sim'"),
              y='comorbidade',
              color = 'C0')
plt.title("Qtd de pessoas contaminadas e comorbidades")
plt.xlabel("Qtd de pessoas contaminadas")
plt.ylabel("Comorbidade")
plt.show()

In [ ]:
sns.barplot(data = casos_explode.query("alguma_comorbidade == 'sim'") \
                                    .melt(id_vars=['municipio_FIOCRUZ','data_FIOCRUZ','obito_pre','idade_pre'],
                                            value_vars=comorbidades,
                                            value_name="existencia_comorbidade",
                                            var_name= "comorbidade") \
                                    .query("existencia_comorbidade == 'sim'") \
                .pivot_table(index='comorbidade',columns="obito_pre",values="existencia_comorbidade",aggfunc="count") \
                .rename(columns = {0:"nao_obito",1:"obito"}).assign(perc = lambda x: x["obito"]/(x["obito"]+x["nao_obito"])) \
                .reset_index().sort_values("perc",ascending=False),
            x = "perc",y = 'comorbidade',color = 'C0')
plt.title("Percentual de mortes e comorbidades")
plt.xlabel("Percentual de óbitos por pessoas contaminadas")
plt.ylabel("Comorbidade")
plt.show()

In [ ]:
relacao = casos_explode.groupby(["obito_pre","alguma_comorbidade"]).agg({"idade_pre":"count"}).reset_index()

#quantidade de obitos com comorbidade frente a todos os obitos
relacao_comorbidade_obito = relacao.query("%s == 'sim' & obito_pre == 1" % "alguma_comorbidade") \
                                    ["idade_pre"].sum() / relacao.query("obito_pre == 1")["idade_pre"].sum()

#quantidade de pessoas com comorbidade frente a todos as pessoas contaminadas
relacao_comorbidade = relacao.query("%s == 'sim'" % "alguma_comorbidade")["idade_pre"].sum()/relacao["idade_pre"].sum()

print("RCO = Relação de comorbidades associadas aos óbitos",round(relacao_comorbidade_obito*100,2),"%")
print("RC = Relação de comorbidades associadas à população contaminada",round(relacao_comorbidade*100,2),"%")
print("Relação RCO/RC",round(relacao_comorbidade_obito/relacao_comorbidade,2))

---
---


In [ ]:
# g = sns.relplot(x=casos_explode.query("obito_pre == 1 & alguma_comorbidade == 'sim'") \
#                                     .melt(id_vars=['municipio_FIOCRUZ','data_FIOCRUZ','obito_pre','idade_pre'],
#                                             value_vars=comorbidades,
#                                             value_name="existencia_comorbidade",
#                                             var_name= "Infecções com comorbidade") \
#                                     .query("existencia_comorbidade == 'sim'")['Infecções com comorbidade'].value_counts(),
#             y= casos_explode.query("obito_pre == 1 & alguma_comorbidade == 'sim'") \
#                                     .melt(id_vars=['municipio_FIOCRUZ','data_FIOCRUZ','obito_pre','idade_pre'],
#                                             value_vars=comorbidades,
#                                             value_name="existencia_comorbidade",
#                                             var_name= "Obitos com comorbidade") \
#                                     .query("existencia_comorbidade == 'sim'")['Obitos com comorbidade'].value_counts())
# x_limits = g.ax.get_xlim()
# y_limits = g.ax.get_ylim()
# min_limit = min(min(x_limits), min(y_limits))
# max_limit = max(max(x_limits), max(y_limits))
# g.ax.plot([min_limit, max_limit], [min_limit, max_limit], color='red', linestyle='--')
# plt.show()

In [ ]:
# # Vamos verificar os casos percentuais, ou seja, a quantidade de casos pela população do municipio
# def regplot_perc_log(df,x,y):
#     sns.regplot(
#         data = df.groupby("municipio_FIOCRUZ").agg({'casosNovos_FIOCRUZ':sum,
#                                                     'obitosNovos_FIOCRUZ':sum,
#                                                     'populacaoTCU2019_FIOCRUZ':'mean'}) \
#                  .assign(perc_casos = lambda x: 100*x["casosNovos_FIOCRUZ"]/x["populacaoTCU2019_FIOCRUZ"],
#                          perc_obitos = lambda x: 100*x["obitosNovos_FIOCRUZ"]/x["populacaoTCU2019_FIOCRUZ"]).sort_values('perc_casos') \
#                 [[x,'populacaoTCU2019_FIOCRUZ']].assign(poplog = lambda x: np.log(x["populacaoTCU2019_FIOCRUZ"])),
#         x = x,
#         y = y
#     )
#     plt.title('Relação entre a quantidade de casos e população')
#     plt.tight_layout()


# print("""Por conta da cidade de São Paulo ser um outlier no número de habitantes, precisamos fazer uma transformação para
# que este problema não seja percebido""")
# regplot_perc_log(casos,'perc_casos','populacaoTCU2019_FIOCRUZ')
# plt.xlabel("Percentual de casos")
# plt.ylabel("População da cidade")
# plt.show()

- Merge com senso atualizado 2022 e comparar com resultados gitlab
- Analisar cidades em que quantidade de idosos, jovens, crianças, adultos é mais alta.
- informações por genero e faixa etaria
- Numero de pacientes por hopital por mes


In [ ]:
df_unica_comorbidade = casos_explode[comorbidades].apply(lambda row: row.eq('sim').sum() == 1, axis=1)
df_unica_comorbidade = casos_explode[df_unica_comorbidade]